In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

In [2]:
import datetime
from datetime import timedelta

In [9]:
pushUrls = []
pushes = []
mergedPushes = []
userPushes = {}

In [10]:
def relatedOrNot(string):
    flag = False
    keywords = ["同志","甲","甲甲","同婚","韓國瑜","韓總","韓國魚","蔡英文","蔡","小英","空心菜","宋楚瑜",\
               "台灣民眾黨","台民","台民黨","時代力量", "時力","背刺力量", "加蔥力量","背刺","加蔥",\
               "黃國昌", "國昌","中共", "五毛","426","支那","維尼","賤畜","大陸",\
                "習維尼","耿爽","中國","國台辦","陸委會","習近平","北京","陸生","網軍","間諜","王立強","吳斯懷",\
               "強國","共產黨","統","統一","統獨","一國","兩制","台獨","臺灣獨立","臺獨","獨派","台灣女性",\
               "台女","臺灣女性","臺女","母豬","CCR","ㄈㄈ尺","自助餐","女權","女森","柯文哲","柯P","柯p",\
               "柯","反送中","港","黑警","港警","香港","黃之鋒","難民法","政治","庇護","眾志"]
    for k in keywords:
        if k in string:
            return True
    return False

In [11]:
def getPushFromBoard(board, amount, maxN):
    i = 0
    page = maxN
    #page = 3783
    li = []
    opts = Options()
    #proxy = "socks5://localhost:9050"
    #opts.add_argument('--proxy-server={}'.format(proxy))
    ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
    opts.add_argument("user-agent={}".format(ua))
    browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver',options=opts)
    browser2 = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver',options=opts)
    
    #browser = webdriver.Chrome()
    #browser2 = webdriver.Chrome()
    '''
    s = browser.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    s2 = browser2.Session()
    s2.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    
    '''
    
    #print("entered page")
    while page >= 248:
        try:
            #print("i: "+str(i))
            browser.get("https://www.ptt.cc/bbs/"+board+"/index"+str(page)+".html")
            browser2.get("https://www.ptt.cc/bbs/"+board+"/index"+str(page)+".html")
            try:
                btn = browser.find_elements_by_class_name("btn-big")[0]
                if btn is not None:
                    btn.click()
            except:
                pass
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            selectTag = '.title>a'

            for ana in soup.select(selectTag):
                if True:
                    if board == "Gossiping":
                        if not relatedOrNot(ana.text):
                            continue
                    pushUrls.append(ana["href"])
                    url2 = "https://www.ptt.cc/"+ana["href"]
                    url2Txt = ana.text
                    #go to the article page
                    if "公告" not in url2Txt:
                        browser2.get(url2)
                        try:
                            btn2 = browser2.find_elements_by_class_name("btn-big")[0]
                            
                            if btn2 is not None:
                                btn2.click()
                        except:
                            pass

                        soup2 = BeautifulSoup(browser2.page_source, 'html.parser')
                        pushesForThisArticle = []
                        users = {}
                        j = 0
                        dateTxt = browser2.find_elements_by_class_name('article-meta-value')[3].text
                        if dateTxt is None or len(dateTxt) <= 0:
                            dateTxt = ""
                        print("page: "+str(page)+", amount: "+str(len(mergedPushes))+", date: "+dateTxt+", "+url2Txt)
                        for p in soup2.select(".push"):
                            spans = p.select("span")
                            if spans is not None and len(spans) >= 4 and spans[1] is not None and spans[1].text is not None and len(spans[1].text) > 0:
                                if spans[1].text not in users.keys():
                                    users[spans[1].text] = [1, [j]]
                                else:
                                    users[spans[1].text][0] += 1
                                    users[spans[1].text][1].append(j)
                                time2 = spans[3].text.split(" ")[len(spans[3].text.split(" "))-1].replace("\n", "")
                                if ":" in time2 and time2.replace(":", "").isnumeric():
                                    row = [str(j),spans[0].text, spans[1].text, spans[2].text[2:], time2]
                                    #print(row)
                                    pushesForThisArticle.append(row)
                                    j += 1
                            #pushes.append(row)
                        #print(users)
                        #處理推文跨行, 根據留言時間差距一分內算同一則推文處理（合併起來）
                        continueFlag = []
                        for u in users:
                            row = []
                            for k in range(len(users[u][1])):
                                flag = False
                                preTime = datetime.datetime.strptime(pushesForThisArticle[users[u][1][k]][4], '%H:%M')
                                if k+1 < len(users[u][1]):
                                    thisTime = datetime.datetime.strptime(pushesForThisArticle[users[u][1][k+1]][4], '%H:%M')
                                    if (thisTime - preTime).seconds <= 60:
                                        flag = True
                                        #print("yes")
                                    else:
                                        pass
                                        #print(str((thisTime - preTime).seconds))
                                        #orThisArticle[u[1][k]][3] += pushesForThisArticle[u[1][k+1]][3]
                                if flag:
                                    row.append(1)
                                else:
                                    row.append(0)
                            lockV = 0
                            for r in range(len(row)):
                                if lockV > 0:
                                    r += 1
                                    lockV -= 1
                                    continue
                                else:
                                    i += 1
                                    content = ""
                                    if row[r] == 0:
                                        content = pushesForThisArticle[users[u][1][r]][3]
                                        mergedPushes.append([str(i), u, content,url2Txt, ana["href"], dateTxt])
                                    else:
                                        content = ""
                                        while row[r] == 1:
                                            content += pushesForThisArticle[users[u][1][r]][3]
                                            #pushesForThisArticle[users[u][1][r]][3] = ""
                                            r += 1
                                            lockV += 1
                                        content += pushesForThisArticle[users[u][1][r]][3]
                                        #print(content+", r: "+str(r))
                                    #if len(content) >= 1:
                                        mergedPushes.append([str(i), u, content,url2Txt, ana["href"], dateTxt])
                                #print(content)
                            #merged_p = ""
                            #for k in range(len(listToMerge)):
                            #    merged_p += pushesForThisArticle[u[1][k]][3]
                                        #mergedPushes.append(str(i+1), u[0], pushesForThisArticle[u[1][k]][3]+\
                                        #                    pushesForThisArticle[u[1][k+1]][3])
                        #print(pushesForThisArticle) 
                else:
                    break
            page -= 1
        except Exception as e:
            print(e)
            #page -= 1
            #time.sleep(5)
            return page
    
    browser.close()
    browser2.close()
    return page

In [ ]:
#Gossiping & HatePolitics各抓6000個
initMax = 3783
nowp = 2645
while nowp >= 248:
    try:
        print("nowp: "+str(nowp))
        page = getPushFromBoard("HatePolitics", 6000, nowp)
        nowp = page -1
    except Exception as e:
        nowp -= 1
        print("e: nowp: "+str(nowp))
        print(e)
        pass

nowp: 2645
page: 2645, amount: 0, date: Wed Dec 18 20:39:49 2019, [黑特] 政見發表會/蔡宣揚政績 韓展現高度
page: 2645, amount: 16, date: Wed Dec 18 20:40:03 2019, [黑特] 蔡不看稿會死阿
page: 2645, amount: 53, date: Wed Dec 18 20:40:44 2019, [討論] 狼人殺邏輯，後置位發言是不是比較有利？


In [7]:
print(len(mergedPushes))

23022


In [8]:
with open('pushesHPall_-2645.csv', 'w', newline='') as csvFile:
  # 建立 CSV 檔寫入器
    writer = csv.writer(csvFile)
    writer.writerow(["num", "id", "content", "article", "articleUrl", "dateTime"])#write header
    for i in range(len(mergedPushes)):
        if str(mergedPushes[i][1]) not in userPushes.keys():
            userPushes[mergedPushes[i][1]] = [str(i+1)]
        else:
            userPushes[mergedPushes[i][1]].append(str(i+1))
        writer.writerow([str(i+1), mergedPushes[i][1],mergedPushes[i][2],mergedPushes[i][3],mergedPushes[i][4],mergedPushes[i][5]])

In [201]:
pushesGS = getPushFromBoard("Gossiping", 6000, 38895)

In [202]:
with open('pushesGSall.csv', 'w', newline='') as csvFile:
  # 建立 CSV 檔寫入器
    writer = csv.writer(csvFile)
    writer.writerow(["num", "id", "content", "article"])#write header
    for i in range(len(pushesGS[1])):
        writer.writerow([str(i+1), pushesGS[1][i][1],pushesGS[1][i][2]])